# はじめに

ハンズオン基礎編は、以下の2つのメニューで構成されています。

## Jupyter Notebook の操作方法

- Jupyter Notebook の基本的な使い方を確認します。

## AWSのクラウドでのサーバ起動

- VCPを利用し、Jupyter Notebook からAWSクラウド上にサーバを起動します。
- 起動したサーバへSSH経由でリモートコマンドを実行し、負荷をかけます。
- サーバの負荷状況を Grafana のダッシュボードから確認します。


# Jupyter Notebook の操作方法

まずはじめに、Jupyter Notebook の基本操作を確認します。

Jupyter NotebookはCellという単位で文書を構築します。
このCellにはPythonコードやMarkdownテキストといった種別があり、適切なものを選択し文書を作成していきます。

Jupyter Notebook のツールバーにある実行ボタンを押す、または、キーボードの Shift + Enter 操作により
Cell を順番に実行していくことができ、Pythonコードが実行されたりMarkdownドキュメントが表示されたりします。
![](media/run-icon.png)

## Python Code を実行する

In [1]:
name = 'Hello world'
print(name.upper())

HELLO WORLD


In [2]:
strings = ['pine', 'apple']
print(' '.join(strings))

pine apple


In [3]:
2+3

5

In [4]:
2**5

32

## Linux コマンドを実行する

In [5]:
!ls -l

total 80
-rw-r--r-- 1 bit_kun users 68905 Jun 26 17:27 00_オンデマンドクラウド構築ハンズオン基礎編.ipynb
-rw-r--r-- 1 bit_kun users   403 Jun 14 13:38 handson_config.py
drwxr-xr-x 3 bit_kun users  4096 Jun 13 15:40 media
drwxr-xr-x 2 bit_kun users  4096 Jun 14 13:50 __pycache__


In [6]:
!mkdir -p xyz/test/aa/bb && touch xyz/test/x.txt && tree xyz
!rm -rf xyz && ls -l

xyz
└── test
    ├── aa
    │   └── bb
    └── x.txt

3 directories, 1 file
total 80
-rw-r--r-- 1 bit_kun users 68905 Jun 26 17:27 00_オンデマンドクラウド構築ハンズオン基礎編.ipynb
-rw-r--r-- 1 bit_kun users   403 Jun 14 13:38 handson_config.py
drwxr-xr-x 3 bit_kun users  4096 Jun 13 15:40 media
drwxr-xr-x 2 bit_kun users  4096 Jun 14 13:50 __pycache__


# AWSクラウド環境でのサーバ作成

オンデマンド構築サービスのVCP SDKを利用して、AWSクラウド環境にサーバを作成します。

![](media/vcp-handson.png)

## VCP SDK 初期化

最初に VCP SDK を初期化します。

In [18]:
from common import logsetting
from vcpsdk.vcpsdk import VcpSDK
import handson_config

#
# VCP SDK の初期化
#
vcc_access_token = handson_config.vcc_access_token

my_vc_name = "vcp_handson"

v = VcpSDK(vcc_access_token, my_vc_name)

# VCP SDK バージョン確認
v.version()


vcplib:
  filename: /notebooks/notebook/vcplib/occtr.py
  version: 18.04.1+201805010

vcpsdk:
  filename: /notebooks/notebook/vcpsdk/vcpsdk.py
  version: 18.04.1+20180510

  plugin:
    aws: 1.0+20180331
    azure: 1.0+20180331
    sakura: 1.0+20180331
    aic: 1.0+20180331
    gcp: 1.0+20180331
    onpremises: 1.0+20180331

server:
    host: 192.168.2.1
    name: VCC514


新規にサーバを作成する前に、SDKを初期化したときのVC状態を確認します。  
ここで出力される `vcid` は実行環境によって異なります。

In [19]:
print(v)

vc_name: vcp_handson

[Vc]
+ name[vcp_handson] vcno(16) state[RUNNING] vcid[8016bd91a026435aa8e0fcd64726d3de]





## 現在起動しているサーバ（が無いこと）を確認

VC `my_vc_name` の上で実行中のサーバを確認します。
まだサーバが1個も無い状態が確認できます。

In [20]:
v._vc.units_df()

,vcno,vcname,vc_state,vcid,cdate,unit_name,unit_state


## SSH キーペアの準備

これから起動するサーバにSSHで接続するための準備を行います。  
パスフレーズ無しのSSH鍵ペアを作成し、公開鍵認証でSSHログインできるようにします。

In [21]:
!ssh-keygen -f id_rsa -N ""

Generating public/private rsa key pair.
Your identification has been saved in id_rsa.
Your public key has been saved in id_rsa.pub.
The key fingerprint is:
d7:f1:b1:e2:c3:4b:4c:f7:ab:3c:39:59:f8:84:97:75 bit_kun@57d8d7164fc4
The key's randomart image is:
+---[RSA 2048]----+
|                 |
|                 |
|            . .  |
|           . o oE|
|        S . + * +|
|         . = = * |
|            * B .|
|           ..B ..|
|            .o+. |
+-----------------+


In [22]:
# SSH公開鍵 `./id_rsa.pub` の内容を1行のBase64形式になるようにエンコードする。
base64_pubkey_tmp = !openssl base64 -e -A < ./id_rsa.pub
base64_pubkey = base64_pubkey_tmp[0]

In [23]:
base64_pubkey

'c3NoLXJzYSBBQUFBQjNOemFDMXljMkVBQUFBREFRQUJBQUFCQVFESk9qMk9VdDdNTDMzU0FFYnhpSDlFQWlwcWhtelN0TDdIekR1R0loUHUwSnorRFc2VGZnMXArbCtKU25jQkZHd2dHbldMRW50SXRMWUhTaDlIUERKOENIdzRNcCttYWJXZHlnUmlteHkvaW80aXJqbzFLNkdHSkFUb3F3MXZ3UnZuZzVQcVczaTZKbnpGSm11ZkN3T0ZseGRIZ1pRaHpqUng5eERGTlJUZU1GQ3Q0MFZncnFmNWp4MlNTZDFJVlJsSEc3K0xSM1h2ZU5Qc21OMTJaeUFreWZXMU12R1JLNGF3aGZSc25IUG1ZaTA2dVF6Vmdxa25kL0NLRHNhcnQrYU1pVU8wL0ZtRjdnbk9HK29jQ016QmJQV25QSWVNVG5UQU5qUXlpYWY4VjhWeHpkZXBkdHJvMDdTSTh4Z0JKOGQwaGhPQis1a1QzNEg3RmtHbStsR0wgYml0X2t1bkA1N2Q4ZDcxNjRmYzQK'

## 新規サーバを作成

サーバの Spec 情報を指定し、 `VcpUnitClass#create` API を呼び出すことでサーバを起動することができます。

### 必須のパラメータ設定
サーバの Spec 情報として、クラウドプロバイダに応じてインスタンスタイプなどが設定済みの Flavor を指定します。

In [24]:
# AWS の small タイプの Flavor を指定
spec = v.spec.find_spec("aws", "small")

# サーバへ公開鍵認証で接続するために、作成済みの鍵情報を指定
spec.params_e = ['AUTHORIZED_KEYS='+base64_pubkey]

### オプションのパラメータ設定
以下の各Spec情報については、オプションとして指定することができます。

In [25]:
# 同じSpecのサーバを複数起動する場合の個数 (デフォルト値: 1)
# spec.num_nodes = 1

# クラウドのインスタンスタイプ名
# spec.instance_type = 't2.medium'

# クラウドのインスタンスの Volume マウント指定 (`docker run -v` オプションに対応)
# spec.params_v = ['/var/lib/docker', '/mnt:/opt']

# クラウドのインスタンスに割り当てるルートディスクサイズ (単位: GB)
# spec.volume_size = 8

# クラウドのインスタンスに割り当てるボリュームタイプ (AWSの選択肢: standard|io1|gp2|sc1|st1)
# spec.volume_type = "standard"

# サーバの固定IPアドレス (プライベートIPアドレス)
# spec.ip_address_list = ['10.0.0.123']

# サーバに使用するコンテナイメージ名 (Baseコンテナ)
# spec.image = 'vcp/base:1.2'   # Baseコンテナイメージ名 

# サーバにアタッチする拡張ディスク (EBSボリュームID)
# spec.volume_id = ['vol-08cbb04b35c8c9545']

# クラウドのインスタンスに設定するタグ情報
# spec.set_tag('key1', 'value1')
# spec.set_tag('key2', 'value2')

### 起動
作成したSpec情報を指定して、 `VcpUnitClass#create` API を呼び出すことでサーバを起動します。

In [26]:
unit_name = 'new_server'
nodes = v.unit.create(unit_name, spec, wait_for=True, verbose=0)

2018-06-26 22:40:52,836 - INFO - BOOTING ... 0 sec
2018-06-26 22:40:58,118 - INFO - BOOTING ... 5 sec
2018-06-26 22:41:03,398 - INFO - BOOTING ... 10 sec
2018-06-26 22:41:08,680 - INFO - BOOTING ... 15 sec
2018-06-26 22:41:13,965 - INFO - BOOTING ... 20 sec
2018-06-26 22:41:19,249 - INFO - BOOTING ... 25 sec
2018-06-26 22:41:24,531 - INFO - BOOTING ... 30 sec
2018-06-26 22:41:29,818 - INFO - BOOTING ... 35 sec
2018-06-26 22:41:35,096 - INFO - BOOTING ... 40 sec
2018-06-26 22:41:40,386 - INFO - BOOTING ... 45 sec
2018-06-26 22:41:45,723 - INFO - BOOTING ... 50 sec
2018-06-26 22:41:51,000 - INFO - BOOTING ... 55 sec
2018-06-26 22:41:56,306 - INFO - BOOTING ... 60 sec
2018-06-26 22:42:01,595 - INFO - BOOTING ... 65 sec
2018-06-26 22:42:06,876 - INFO - BOOTING ... 70 sec
2018-06-26 22:42:12,174 - INFO - unit new_server is RUNNING


## 起動したサーバ情報を確認

サーバの起動状態、IPアドレス、クラウドインスタンスIDなどを確認します。

In [27]:
for node in nodes:
    print(node)
    for vol in node.volumes:
        print(vol)

[VcNode]
+ no(1) state[RUNNING] id[9bff0d9743204ad9820fd7a57a768168] cloud_instance_address[172.30.2.111] cloud_instance_id[i-0707b6340c5b92b7a]



In [28]:
for node in v.unit.find_node(unit_name="new_server"):
    print(node.state)
    print(node.cloud_instance_address)

RUNNING
172.30.2.111


In [29]:
# VC に含まれる Unit 情報
v._vc.units_df()

,vcno,vcname,vc_state,vcid,cdate,unit_name,unit_state
0,16,vcp_handson,RUNNING,8016bd91...,2018/06/26 13:39:08 UTC,new_server,RUNNING


In [30]:
# Unit に含まれる Node 情報
v._vc.units[0].nodes_df()

,vcno,vc_name,unit_name,unit_state,node_no,node_id,node_state,cloud_instance_address,cloud_instance_id,cloud_instance_name,volumes
0,16,vcp_handson,new_server,RUNNING,1,9bff0d97...,RUNNING,172.30.2.111,i-0707b6340c5b92b7a,VCP-38f612b6-8016bd91,none


## 起動したサーバにSSHリモートコマンド実行

起動したサーバで、Applicationコンテナを1個起動します。  
今回は、stress コマンドを含むコンテナ・イメージを利用してサーバに負荷をかけてみることにします。

![](media/vcp-handson2.png)

まず最初に、公開鍵認証によるSSH接続でリモートコマンド実行ができることを確認します。

In [31]:
# SSHで `hostname` コマンドを実行
server = v.unit.find_node(unit_name="new_server")[0].cloud_instance_address
!ssh -i ./id_rsa -o StrictHostKeyChecking=no root@$server hostname

ip-172-30-2-111


SSH接続が可能なことを確認できたら、stress コマンドを含むコンテナ・イメージを起動します。  
以下の例では、60秒間だけ CPU x 2 とメモリ 128 MB を消費します。

In [32]:
# SSHで `docker run` コマンドを実行し、stress コンテナを起動
stress_cmd = '''\
/usr/local/bin/docker run --rm --name stress polinux/stress \
  stress --cpu 2 --io 1 --vm 2 --vm-bytes 128M --timeout 60s --verbose \
  | tail -n10
'''
!ssh -i ./id_rsa -o StrictHostKeyChecking=no root@$server $stress_cmd

Unable to find image 'polinux/stress:latest' locally
latest: Pulling from polinux/stress
627beaf3eaaf: Pulling fs layer
4de9f469a6f4: Pulling fs layer
627beaf3eaaf: Verifying Checksum
627beaf3eaaf: Download complete
4de9f469a6f4: Verifying Checksum
4de9f469a6f4: Download complete
627beaf3eaaf: Pull complete
4de9f469a6f4: Pull complete
Digest: sha256:6d1825288ddb6b3cec8d3ac8a488c8ec2449334512ecb938483fc2b25cbbdb9a
Status: Downloaded newer image for polinux/stress:latest
stress: dbug: [7] touching bytes in strides of 4096 bytes ...
stress: dbug: [9] freed 134217728 bytes
stress: dbug: [9] allocating 134217728 bytes ...
stress: dbug: [9] touching bytes in strides of 4096 bytes ...
stress: dbug: [1] <-- worker 5 signalled normally
stress: dbug: [1] <-- worker 8 signalled normally
stress: dbug: [1] <-- worker 9 signalled normally
stress: dbug: [1] <-- worker 7 signalled normally
stress: dbug: [1] <-- worker 6 signalled normally
stress: info: [1] successful run completed in 60s


# Grafanaダッシュボードで負荷状況を見る


## Grafanaへのログイン

ユーザ名とパスワードは両方とも `admin` です。  
⇒[**ログイン**](/grafana/d/handson/vcp-metrics?refresh=30s&orgId=1)

![](media/grafana-login.png)

## 負荷状況のグラフ表示

以下のように表示されます。

![](media/grafana-metrics.png)

# 使い終わったサーバの削除

VCP SDK の cleanup API を呼び出します。この結果、上記手順で起動した全サーバが削除されます。

In [33]:
print(v)

vc_name: vcp_handson

[Vc]
+ name[vcp_handson] vcno(16) state[RUNNING] vcid[8016bd91a026435aa8e0fcd64726d3de]





In [34]:
v.cleanup()

2018-06-26 22:43:59,973 - INFO - new_server is DELETING ... 0 sec
2018-06-26 22:44:05,299 - INFO - new_server is DELETING ... 5 sec
2018-06-26 22:44:10,619 - INFO - new_server is DELETING ... 10 sec
2018-06-26 22:44:15,945 - INFO - new_server is DELETING ... 15 sec
2018-06-26 22:44:21,264 - INFO - new_server is DELETING ... 20 sec
2018-06-26 22:44:26,585 - INFO - new_server is DELETING ... 25 sec
2018-06-26 22:44:31,900 - INFO - new_server is DELETING ... 30 sec
2018-06-26 22:44:37,215 - INFO - new_server is DELETING ... 35 sec
2018-06-26 22:44:42,532 - INFO - new_server is DELETING ... 40 sec
2018-06-26 22:44:47,856 - INFO - new_server is DELETING ... 45 sec
2018-06-26 22:44:53,026 - INFO - cleanup completed. vc vcp_handson is cleanup(no unit)


正しく削除されたことを確認します。１つもサーバ情報が出力されなければ削除が成功しています。

In [35]:
print(v)

vc_name: vcp_handson

no unit



# （参考資料）
## クラウド環境構築テンプレート - Jupyter Notebook によるノウハウ共有

[オープンフォーラム2017：【クラウドトラック】](http://www.nii.ac.jp/csi/openforum2017/track/day2_4.html)

「クラウド環境構築テンプレート Jupyter Notebook によるノウハウ共有」
 * http://www.nii.ac.jp/csi/openforum2017/track/pdf/20170608PM_C_02_masatani.pdf